In [250]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.by import By
import json
import pandas as pd

# Initialize a list to store the data for each individual page
all_data = {}

# Initialize the web driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# Initial search page URL
url = "https://samlingar.shm.se/sok?type=object&query=Vikingatid&rows=1000&offset=0"
driver.get(url)

# Wait for the page to load (you may need to adjust the sleep duration)
time.sleep(2)

# Find the cookie disclaimer button by its aria-label
cookie_disclaimer = driver.find_element(By.CSS_SELECTOR, '[aria-label="Godkänn alla kakor"]')
# Check if the cookie disclaimer button is displayed and then click it
if cookie_disclaimer.is_displayed():
    ActionChains(driver).move_to_element(cookie_disclaimer).click().perform()

# Find all the search result items
results = driver.find_elements(By.XPATH, "//a[contains(@class,'rKAserSVOTC2Sy4lh50nJg') and contains(@class,'miFVghdNq-ew7wuzMJWdDw')]")
from collections import defaultdict

# Dictionary to hold the title and its corresponding URL
titles_and_urls = defaultdict(list)

for result in results:
    # Extract the aria-label attribute and split it to get the title
    aria_label = result.get_attribute('aria-label')
    title_split = aria_label.split(' - Föremålsbenämning:')
    title = title_split[0].strip()

    # Extract the URL
    url = result.get_attribute('href')

    # Modify title if it's a duplicate
    if titles_and_urls[title]:
        count = len(titles_and_urls[title]) + 1
        title = f"{title} ({count})"
    
    # Add the title and URL to the dictionary
    titles_and_urls[title].append(url)

# Flatten the dictionary to ensure each title has a unique URL
titles_and_urls = {k: v[0] for k, v in titles_and_urls.items()}

# Dictionary to hold the data for all items
all_items_dict = {}

# Iterate over titles and URLs to fetch table data
for title, url in titles_and_urls.items():
    # Open the link of the result
    driver.get(url)
    time.sleep(3)  # wait for the page to load

    # Grab the first table on the page as the data source
    try:
        table = pd.read_html(driver.page_source)[0]
        row_dict = {row[0]: row[1] for row in table.itertuples(index=False)}
        all_items_dict[title] = row_dict
    except IndexError:
        print(f"No table found on page for {title}")
    except Exception as e:
        print(f"Error processing table for {title}: {e}")

driver.quit()
# You now have a dictionary with item titles as keys and table data (as dictionaries) as values
all_data_json = json.dumps(all_items_dict, indent=4)

# Convert the JSON string to a Python dictionary
data_dict = json.loads(all_data_json)

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data_dict, orient='index')

# Reset the index to make the item titles a column
df = df.reset_index()

# Rename the former index column to 'unique_name'
df = df.rename(columns={'index': 'unique_name'})
df.to_csv('Viking_artifacts.csv')